In [1]:
import pandas as pd
from sqlalchemy import (
    create_engine, MetaData, Table, Column,
    String, Integer, Float, DateTime, Boolean, ForeignKey, text
)
DB_URL = "sqlite:///crime.db"
engine = create_engine(DB_URL, future=True)
metadata = MetaData()
metadata.reflect(bind=engine)
metadata.drop_all(bind=engine)
metadata = MetaData()

- create main table

In [2]:
PK = 'id'
main = pd.read_csv("event.csv")
complaints = pd.read_csv("complaint.csv")
arrests = pd.read_csv("arrest.csv")
shootings = pd.read_csv("shooting.csv")
main["id"] = pd.to_numeric(main["id"], errors="coerce").astype("Int64")
main["nypdCode"] = pd.to_numeric(main["nypdCode"], errors="coerce").astype("Int64")
main["precinct"] = pd.to_numeric(main["precinct"], errors="coerce").astype("Int64")
main["jurisdictionCode"] = pd.to_numeric(main["jurisdictionCode"], errors="coerce").astype("Int64")
main["offenseCode"] = pd.to_numeric(main["offenseCode"], errors="coerce").astype("Int64")
main["latitude"] = pd.to_numeric(main["latitude"], errors="coerce")
main["longitude"] = pd.to_numeric(main["longitude"], errors="coerce")
complaints["id"] = pd.to_numeric(complaints["id"], errors="coerce").astype("Int64")
complaints["start_datetime"] = pd.to_datetime(complaints["start_datetime"], errors="coerce")
complaints["end_datetime"] = pd.to_datetime(complaints["end_datetime"], errors="coerce")
arrests["id"] = pd.to_numeric(arrests["id"], errors="coerce").astype("Int64")
arrests["keyCode"] = pd.to_numeric(arrests["keyCode"], errors="coerce").astype("Int64")
arrests["xCoord"] = pd.to_numeric(arrests["xCoord"], errors="coerce")
arrests["yCoord"] = pd.to_numeric(arrests["yCoord"], errors="coerce")
shootings["id"] = pd.to_numeric(shootings["id"], errors="coerce").astype("Int64")
shootings["xCoord"] = pd.to_numeric(shootings["xCoord"], errors="coerce")
shootings["yCoord"] = pd.to_numeric(shootings["yCoord"], errors="coerce")
main_table = Table(
    "main",
    metadata,
    Column("id", Integer, primary_key=True),                 
    Column("is_complaint", Integer, nullable=False),
    Column("is_arrest", Integer, nullable=False),
    Column("is_shooting", Integer, nullable=False),                 
    Column("dateId", DateTime, nullable=True),                 
    Column("borough", String, nullable=True),                
    Column("nypdCode", Integer, nullable=True),
    Column("precinct", Integer, nullable=True),              
    Column("latitude", Float, nullable=True),
    Column("longitude", Float, nullable=True),               
    Column("jurisdictionCode", Integer, nullable=True),      
    Column("offenseCode", Integer, nullable=True),           
)
complaints_table = Table(
    "complaints",
    metadata,
    Column("id", Integer, ForeignKey('main.id'),primary_key=True),
    Column("start_datetime", DateTime, nullable = True),
    Column("end_datetime", DateTime, nullable = True),
    Column("reportDate", String, nullable = True),
    Column("crimeStatus", String, nullable = True),
    Column("jurisdictionDescription", String, nullable = True),
    Column("lawCategory", String, nullable = True),
    Column("spatialContext", String, nullable = True),
    Column("offenseDescription", String, nullable = True),
    Column("policeDescription", String, nullable = True),
    Column("premisesDescription", String, nullable = True),
    Column("suspAgeGroup", String, nullable = True),
    Column("suspRace", String, nullable = True),
    Column("suspSex", String, nullable = True),
    Column("vicAgeGroup", String, nullable = True),
    Column("vicRace", String, nullable = True),
    Column("vicSex", String, nullable = True),   
)

arrest_table = Table(
    "arrests",
    metadata,
    Column("id", Integer, ForeignKey('main.id'),primary_key=True),
    Column("lawCategory", String, nullable=True),
    Column("offenseDescription", String, nullable=True),
    Column("policeDescription", String, nullable=True),
    Column("keyCode", Integer, nullable=True),
    Column("lawCode", String, nullable=True),
    Column("perpAgeGroup", String, nullable=True),
    Column("perpSex", String, nullable=True),
    Column("perpRace", String, nullable=True),
    Column("xCoord", Float, nullable=True),
    Column("yCoord", Float, nullable=True),
)
shooting_table = Table(
    "shootings",
    metadata,
    Column("shooting_record_id", Integer, primary_key=True, autoincrement=True),
    Column("id", Integer, ForeignKey("main.id"), nullable=False),
    Column("time", String, nullable=True),
    Column("spatialContext", String, nullable=True),
    Column("locationType", String, nullable=True),
    Column("venueType", String, nullable=True),
    Column("vicAgeGroup", String, nullable=True),
    Column("vicRace", String, nullable=True),
    Column("vicSex", String, nullable=True),
    Column("perpAgeGroup", String, nullable=True),
    Column("perpSex", String, nullable=True),
    Column("perpRace", String, nullable=True),
    Column("xCoord", Float, nullable=True),
    Column("yCoord", Float, nullable=True),
    Column("fatalFlag", Boolean, nullable=True),    
)
    

metadata.create_all(engine)
with engine.begin() as conn:
    conn.execute(text("PRAGMA foreign_keys=ON"))


/tmp/ipykernel_3308/3975712541.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  arrests = pd.read_csv("arrest.csv")


In [3]:

main.to_sql("main", engine, if_exists="append", index=False)
complaints.to_sql("complaints", engine, if_exists="append", index=False)
arrests.to_sql("arrests", engine, if_exists="append", index=False)
shootings.to_sql("shootings", engine, if_exists="append", index=False)

170

In [4]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text(
        "SELECT name FROM sqlite_master WHERE type='table';"
    ))

    for row in result:
        print(row)

('main',)
('complaints',)
('arrests',)
('shootings',)


In [5]:
with engine.connect() as conn:
    result = conn.execute(text(
        "SELECT * FROM main m JOIN shootings s ON m.id = s.id WHERE m.borough = 'MANHATTAN';"
    ))

    for row in result:
        print(row)

(298909833, 0, 0, 1, '1/4/2025', 'MANHATTAN', 0, 28, 40.802702, -73.95263, 0, 0, 20, 298909833, '16:50:00', 'OUTSIDE', 'STREET', 'UNKNOWN', '25-44', 'BLACK', 'M', '18-24', 'M', 'BLACK', 997365.0, 231723.0, 0)
(298949944, 0, 0, 1, '1/5/2025', 'MANHATTAN', 0, 24, 40.791193, -73.972463, 0, 0, 27, 298949944, '19:43:00', 'OUTSIDE', 'STREET', 'UNKNOWN', '<18', 'BLACK', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 991875.0, 227527.0, 0)
(299382241, 0, 0, 1, '1/13/2025', 'MANHATTAN', 0, 30, 40.824207, -73.95208, 0, 0, 35, 299382241, '21:15:00', 'OUTSIDE', 'STREET', 'UNKNOWN', '25-44', 'BLACK HISPANIC', 'M', '18-24', 'M', 'BLACK HISPANIC', 997512.0, 239558.0, 0)
(299382241, 0, 0, 1, '1/13/2025', 'MANHATTAN', 0, 30, 40.824207, -73.95208, 0, 0, 36, 299382241, '21:15:00', 'OUTSIDE', 'STREET', 'UNKNOWN', '25-44', 'BLACK HISPANIC', 'M', '18-24', 'M', 'BLACK', 997512.0, 239558.0, 0)
(299382241, 0, 0, 1, '1/13/2025', 'MANHATTAN', 0, 30, 40.824207, -73.95208, 0, 0, 37, 299382241, '21:15:00', 'OUTSIDE', 'STREE